 # Importación de paquetes iniciales

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import logging
import os


 # Configuración del sistema de excepciones y logs
 Se van a almacenar en una carpeta en el root según las indicaciones de la prueba

In [ ]:
# Crear un directorio de registros si no existe
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# Ajustar la configuración de registro
log_file = os.path.join(log_dir, "application_limpieza.log")
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(lineno)s - %(message)s"
)
file_handler = logging.FileHandler(log_file, "w")
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(
    logging.Formatter("%(asctime)s - %(levelname)s - %(lineno)s - %(message)s")
)
logging.getLogger().addHandler(file_handler)

logging.debug("Prueba de mensaje de depuración")
logging.info("Prueba de mensaje informativo")
logging.warning("Prueba de mensaje de advertencia")
logging.error("Prueba de mensaje de error")


In [ ]:
# Define la ruta del archivo
ruta_archivo = "datasets/flights.csv"

# Lee el archivo CSV en un DataFrame de pandas, por velocidad se hace con una muestra de 400.000 filas
df = pd.read_csv(ruta_archivo, delimiter="|", nrows=400000)

# Muestra las primeras filas del DataFrame
df.head()


In [ ]:
df.shape


 Se comprueba que el dataset tiene más de un millon de filas como se pide en la prueba y que carga todas las filas del csv.

 # EDA

In [ ]:
# Se revisa el tipo de cada columna para identificar si están correctas o se debe hacer algun ajuste
df.info()


 Se identifica que la columna FlightDate deberia ser una fecha, la distancia debería ser un número, Cancelled y Diverted deben tener valor booleano.

In [ ]:
try:
    # Se convierte la fecha
    df["FLIGHTDATE"] = pd.to_datetime(df["FLIGHTDATE"], format="%Y%m%d")

    # Se ajusta la columna Distance
    df["DISTANCE"] = df["DISTANCE"].str.replace(" miles", "")
    df["DISTANCE"] = pd.to_numeric(df["DISTANCE"], errors="raise")

    df["DEPDELAY"] = pd.to_timedelta(df["DEPDELAY"], unit="min", errors="ignore")
    df["TAXIOUT"] = pd.to_timedelta(df["TAXIOUT"], unit="min", errors="ignore")
    df["TAXIIN"] = pd.to_timedelta(df["TAXIIN"], unit="min", errors="ignore")
    df["ARRDELAY"] = pd.to_timedelta(df["ARRDELAY"], unit="min", errors="ignore")
    df["CRSELAPSEDTIME"] = pd.to_timedelta(
        df["CRSELAPSEDTIME"], unit="min", errors="ignore"
    )
    df["ACTUALELAPSEDTIME"] = pd.to_timedelta(
        df["ACTUALELAPSEDTIME"], unit="min", errors="ignore"
    )

    # Se ajustan las columnas booleanas
    df["CANCELLED"] = df["CANCELLED"].map(
        {"0": False, "1": True, "True": True, "False": False, "T": True, "F": False}
    )
    df["DIVERTED"] = df["DIVERTED"].map(
        {"0": False, "1": True, "True": True, "False": False, "T": True, "F": False}
    )

except Exception as e:
    logging.exception(f"Se ha presentado una excepcion: {e}")


 Aunque no es del todo necesario capturas los errores en este punto pues es muy manual este proceso se deja para probar el sistema logging, si se corre dos veces este bloque se dispara una excepcion por aplicarle .str. a la columna distance que ya es numerica.

 Para seguir con las columnas que tiene formato de hora vemos que dos de ellas tienen formato int64 y el resto float64, quiere decir que las int no tienen valores nulos mientras que las otras si, se verifica esto para proceder a su transformació.

In [ ]:
# Convertir columnas a formato fecha hora
columnas_hora = [
    "CRSDEPTIME",
    "DEPTIME",
    "WHEELSOFF",
    "WHEELSON",
    "CRSARRTIME",
    "ARRTIME",
]
for columna in columnas_hora:
    if df[columna].isnull().sum() == 0:
        df[columna] = df[columna].astype(str).str.zfill(4)
        df[columna] = pd.to_datetime(
            df[columna], errors="coerce", format="%H%M"
        ).dt.time
        df[columna] = (
            df["FLIGHTDATE"].dt.strftime("%Y-%m-%d") + " " + df[columna].apply(str)
        )
        df[columna] = pd.to_datetime(df[columna], errors="coerce")


In [ ]:
# Se verifican los cambios
df.info()


 ## Datos faltantes

In [ ]:
# Análisis de valores nulos
null_values = df.isnull().sum() / len(df) * 100
print("Porcentaje de valores nulos por columna:")
print(null_values)


 El porcentaje de datos faltantes en algunos campos va desde 0.9% hasta 15% en otros, en este punto del proceso no se considera eliminar estos registros, se procede a explorar los datos con el fin de decidir si descartar las columnas con alto porcentaje o si utilizar alguna técnica para completar la información faltante.

In [ ]:
# Matriz de correlación
correlation_matrix = df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="RdYlGn")
plt.title("Matriz de correlación")
plt.show()


 Aunque la matriz de correlación se utiliza más para modelos numéricos y preparar la data para modelos predictivos en este punto nos puede dar una pista si al descartar una columna podríamos afectar otra.

 Empezando por la columna que más datos le faltan (TAILNUM) se analiza si es necesario rellenar o no estos valores, esta columna corresponde a la identificación única de cada avión y sin más datos que nos permitan identificar a cada aeronave es imposible conseguir este valor; también se puede apreciar en la matriz que este valor no tiene gran influencia sobre las otras columnas, por lo tanto se dejaran los valores nulos y no se eliminaran estos registros.

 Ahora se procede a revisar ORIGINSTATE y ORIGINSTATENAME, como tenemos el código del aeropuerto podemos buscarlo en los aeropuertos de destino y así encontrar la información faltante

In [ ]:
# Crear un diccionario de mapeo entre DESTAIRPORTCODE y DESTSTATE
codigos = df.set_index("DESTAIRPORTCODE")["DESTSTATE"].to_dict()

# Rellenar los valores faltantes en ORIGINSTATE usando el diccionario de mapeo
df["ORIGINSTATE"] = df["ORIGINSTATE"].fillna(df["ORIGINAIRPORTCODE"].map(codigos))


 Como esta estrategia no redujo el número de datos faltantes se procede a conectarse a una API gratuita que retorna la información del aeropuerto basado en el código único

In [ ]:
def airport_info(airport_code):
    url = "https://airport-info.p.rapidapi.com/airport"

    querystring = {"iata": airport_code}

    headers = {
        "X-RapidAPI-Key": "6f294ed6f0mshd9fbb45d9c15ffbp112336jsn9ed08e8c22a6",
        "X-RapidAPI-Host": "airport-info.p.rapidapi.com",
    }
    try:
        response = requests.get(url, headers=headers, params=querystring)

    except Exception as e:
        logging.exception(f"Se ha presentado una excepcion al consultar la API: {e}")
        return

    print(response.json())

    return response.json()



In [ ]:
# Filtrar el diccionario para obtener solo las claves con valor NaN
codigos_faltantes = {key: value for key, value in codigos.items() if pd.isna(value)}

codigos_completos = {}

for key in codigos_faltantes:
    codigos_completos[key] = airport_info(key)["state"]


 Como ya se tiene un diccionario con el nombre del estado para cada aeropuerto faltante se agregan los datos al dataframe

In [ ]:
df["ORIGINSTATENAME"] = df["ORIGINSTATENAME"].fillna(
    df["ORIGINAIRPORTCODE"].map(codigos_completos)
)
df["DESTSTATENAME"] = df["DESTSTATENAME"].fillna(
    df["DESTAIRPORTCODE"].map(codigos_completos)
)


 Ahora se completan las columnas ORIGINSTATE Y DESTSTATE, para esto se utiliza otro dataset pequeño con todos los estados y su abreviatura

In [ ]:
# Se carga el dataset
df_estados = pd.read_csv("datasets/us_states.tsv", sep="\t")

# Se extraen los estados y su abreviatura
abreviaturas = df_estados.set_index("name")["state"].to_dict()

# Se elimina el dataset para liberar memoria
del df_estados


In [ ]:
df["ORIGINSTATE"] = df["ORIGINSTATE"].fillna(df["ORIGINSTATENAME"].map(abreviaturas))
df["DESTSTATE"] = df["DESTSTATE"].fillna(df["DESTSTATENAME"].map(abreviaturas))


 ### Columnas de tiempo

 Se comprueban si aún quedan valores nulos

In [ ]:
# Análisis de valores nulos
null_values = df.isnull().sum() / len(df) * 100
print("Porcentaje de valores nulos por columna:")
print(null_values)


 Para los valores nulos que aún quedan se van a insertan los valores aproximados, por ejemplo hora de despegue programada y si la real es nula se pondrá la programada para poder rellenar los espacios, de todas formas como se vio al inicio estos valores solo corresponden al 2% y no presentan una cantidad estadisticamente considerable

In [ ]:
df.info()


In [ ]:
# Convertir columnas a formato fecha hora
columnas_hora = ["DEPTIME"]
for columna in columnas_hora:
    df[columna] = df[columna].astype("Int64").astype(str)
    df[columna] = df[columna].fillna(df["CRSDEPTIME"])
    df[columna] = df[columna].astype(str).str.zfill(4)
    df[columna] = pd.to_datetime(df[columna], errors="coerce", format="%H%M").dt.time
    df[columna] = (
        df["FLIGHTDATE"].dt.strftime("%Y-%m-%d") + " " + df[columna].apply(str)
    )
    df[columna] = pd.to_datetime(df[columna], errors="coerce")
    df.loc[(df[columna].isnull()) & (df["DEPDELAY"].isnull()), columna] = df[
        "CRSDEPTIME"
    ]
    df.loc[(df[columna].isnull()) & (df["DEPDELAY"].notnull()), columna] = (
        df["CRSDEPTIME"] + df["DEPDELAY"]
    )

columnas_hora = ["ARRTIME"]
for columna in columnas_hora:
    df[columna] = df[columna].astype("Int64").astype(str)
    df[columna] = df[columna].astype(str).str.zfill(4)
    df[columna] = pd.to_datetime(df[columna], errors="coerce", format="%H%M").dt.time
    df[columna] = (
        df["FLIGHTDATE"].dt.strftime("%Y-%m-%d") + " " + df[columna].apply(str)
    )
    df[columna] = pd.to_datetime(df[columna], errors="coerce")
    df.loc[(df[columna].isnull()) & (df["ARRDELAY"].isnull()), columna] = df[
        "CRSARRTIME"
    ]
    df.loc[(df[columna].isnull()) & (df["ARRDELAY"].notnull()), columna] = (
        df["CRSARRTIME"] + df["ARRDELAY"]
    )


columnas_hora = ["WHEELSOFF"]
for columna in columnas_hora:
    df[columna] = df[columna].astype("Int64").astype(str)
    df[columna] = df[columna].astype(str).str.zfill(4)
    df[columna] = pd.to_datetime(df[columna], errors="coerce", format="%H%M").dt.time
    df[columna] = (
        df["FLIGHTDATE"].dt.strftime("%Y-%m-%d") + " " + df[columna].apply(str)
    )
    df[columna] = pd.to_datetime(df[columna], errors="coerce")
    df.loc[df[columna].isnull(), columna] = df["DEPTIME"] + df["TAXIOUT"]

columnas_hora = ["WHEELSON"]
for columna in columnas_hora:
    df[columna] = df[columna].astype("Int64").astype(str)
    df[columna] = df[columna].astype(str).str.zfill(4)
    df[columna] = pd.to_datetime(df[columna], errors="coerce", format="%H%M").dt.time
    df[columna] = (
        df["FLIGHTDATE"].dt.strftime("%Y-%m-%d") + " " + df[columna].apply(str)
    )
    df[columna] = pd.to_datetime(df[columna], errors="coerce")
    df.loc[df[columna].isnull(), columna] = df["ARRTIME"] - df["TAXIIN"]

df["DEPDELAY"] = df["DEPDELAY"].dt.total_seconds() / 60
df["TAXIOUT"] = df["TAXIOUT"].dt.total_seconds() / 60
df["TAXIIN"] = df["TAXIIN"].dt.total_seconds() / 60
df["ARRDELAY"] = df["ARRDELAY"].dt.total_seconds() / 60
df["CRSELAPSEDTIME"] = df["CRSELAPSEDTIME"].dt.total_seconds() / 60
df["ACTUALELAPSEDTIME"] = df["ACTUALELAPSEDTIME"].dt.total_seconds() / 60


 Se corrigen las fechas de llegada para aquellos aviones que salen un día y llegan al siguiente.

In [ ]:
def arreglar_fecha(row):
    if row["WHEELSOFF"] < row["CRSDEPTIME"]:
        row["WHEELSOFF"] = row["WHEELSOFF"] + pd.to_timedelta(
            1, unit="day", errors="ignore"
        )
        row["WHEELSON"] = row["WHEELSON"] + pd.to_timedelta(
            1, unit="day", errors="ignore"
        )
        row["CRSARRTIME"] = row["CRSARRTIME"] + pd.to_timedelta(
            1, unit="day", errors="ignore"
        )
        row["ARRTIME"] = row["ARRTIME"] + pd.to_timedelta(
            1, unit="day", errors="ignore"
        )
        return row
    elif row["WHEELSON"] < row["WHEELSOFF"]:
        row["WHEELSON"] = row["WHEELSON"] + pd.to_timedelta(
            1, unit="day", errors="ignore"
        )
        row["CRSARRTIME"] = row["CRSARRTIME"] + pd.to_timedelta(
            1, unit="day", errors="ignore"
        )
        row["ARRTIME"] = row["ARRTIME"] + pd.to_timedelta(
            1, unit="day", errors="ignore"
        )
        return row
    elif row["ARRTIME"] < row["WHEELSON"]:
        row["CRSARRTIME"] = row["CRSARRTIME"] + pd.to_timedelta(
            1, unit="day", errors="ignore"
        )
        row["ARRTIME"] = row["ARRTIME"] + pd.to_timedelta(
            1, unit="day", errors="ignore"
        )
        return row
    elif row["ARRTIME"] < row["WHEELSOFF"]:
        row["ARRTIME"] = row["ARRTIME"] + pd.to_timedelta(
            1, unit="day", errors="ignore"
        )
        return row
    elif row["CRSARRTIME"] < row["WHEELSOFF"]:
        row["CRSARRTIME"] = row["CRSARRTIME"] + pd.to_timedelta(
            1, unit="day", errors="ignore"
        )
        return row
    elif row["ARRTIME"] < row["DEPTIME"]:
        row["ARRTIME"] = row["ARRTIME"] + pd.to_timedelta(
            1, unit="day", errors="ignore"
        )

    return row



In [ ]:
df = df.apply(lambda row: arreglar_fecha(row), axis=1)


 Ya con las fechas ajustadas se procede a llenar los vacios en las columnas ACTUALELAPSEDTIME y CRSELAPSEDTIME

In [ ]:
df.loc[df["CRSELAPSEDTIME"].isnull(), "CRSELAPSEDTIME"] = (
    pd.to_timedelta(
        df["CRSARRTIME"] - df["CRSDEPTIME"], unit="min", errors="ignore"
    ).dt.total_seconds()
    / 60
)
df.loc[df["ACTUALELAPSEDTIME"].isnull(), "ACTUALELAPSEDTIME"] = (
    pd.to_timedelta(
        df["ARRTIME"] - df["DEPTIME"], unit="min", errors="ignore"
    ).dt.total_seconds()
    / 60
)


In [ ]:
df.info()


 Se revisa ahora si quedaron vuelos con duración negativa, es posible si los datos originales están corruptos

In [ ]:
# Análisis de valores negativos
df.loc[
    (df["ACTUALELAPSEDTIME"] < 0) | (df["CRSELAPSEDTIME"] < 0), "ACTUALELAPSEDTIME"
].sum()


 Todas las duraciones de los vuelos quedaron bien, sin embargo, con fines demostrativos se realiza un análisis de datos atípicos respecto a la duración y distancia de los para determinar si es necesarios eliminar algunos valores

In [ ]:
# Gráficas de distribución de columnas numéricas
plt.figure(figsize=(12, 10))
for i, columna in enumerate(["ACTUALELAPSEDTIME", "DISTANCE"], 1):
    plt.subplot(4, 3, i)
    sns.histplot(data=df, x=columna, kde=True)
    plt.title(f"Distribución de {columna}")
    plt.xlabel(columna)
    plt.ylabel("Frecuencia")
plt.tight_layout()
plt.show()


In [ ]:
# Análisis de valores atípicos
atipicos = pd.DataFrame(columns=["Columna", "Valor atípico"])

# Para identificar los valores atipicos se utiliza el método del rango intercuartil, si conocieramos mejor el contexto
# se podría escoger otro método como la media movil u otro método
for columna in ["ACTUALELAPSEDTIME", "DISTANCE"]:
    q1 = df[columna].quantile(0.25)
    q3 = df[columna].quantile(0.75)
    iqr = q3 - q1
    limite_inferior = q1 - 1.5 * iqr
    limite_superior = q3 + 1.5 * iqr

    columna_atipica = df[
        (df[columna] < limite_inferior) | (df[columna] > limite_superior)
    ]
    atipicos = pd.concat([atipicos, columna_atipica[[columna]]])

# Se eliminan los valores atípicos
df = df[~df.index.isin(atipicos.index)]

print("Valores atípicos:")
print(atipicos)


 Efectivamente los datos ya tienen una mejor distribución pero como se aclaró anteriormente esto se hace con fines demostrativos, en un caso real se debería tener el contexto de la información para entender estos valores atípicos

In [ ]:
# Gráficas de distribución de columnas numéricas
plt.figure(figsize=(12, 10))
for i, columna in enumerate(["ACTUALELAPSEDTIME", "DISTANCE"], 1):
    plt.subplot(4, 3, i)
    sns.histplot(data=df, x=columna, kde=True)
    plt.title(f"Distribución de {columna}")
    plt.xlabel(columna)
    plt.ylabel("Frecuencia")
plt.tight_layout()
plt.show()


 Con esto termina el proceso de limpieza del dataset para proceder con la carga y diseño de la BD relacional

In [ ]:
df.to_parquet("datasets/flights_clean.parquet", engine="auto")
del df


 El dataset limpio se guarda en formato parquet que es más eficiente y pesa mucho menos. Se puede correr el notebook para generarlo nuevamente